# 泰坦尼克号乘客生存情况预测分析
## 第二部分 特征工程

---
  
　ID：wlong
　数据分析爱好者、目前小白一枚
　邮箱：wlong826@163.com
　CSDN：**[【数分小白龙】](https://blog.csdn.net/qq_38230663)**
　如果有不完善的地方，欢迎小伙伴评论区留言！
　最每周都会有所更新，欢迎大家**关注点赞分享fork**哦！
 
---

　刚入门对Pandas感兴趣的小伙伴，看这👉：[【Pandas数据处理120题知识点总结】](https://www.heywhale.com/mw/project/60c0b06919d601001898394c)
 
　如果有想学习爬虫的小伙伴，看这👉：[【零基础入门Python爬虫--豆瓣、58同城、安居客等】](https://www.heywhale.com/mw/project/60be1fa8acdcb3001707b133)

- 本文的参考项目:

    - [Kaggle之泰坦尼克号问题](https://www.heywhale.com/mw/project/5e898c2295b029002ca8d8ce)
		
    - [Kaggle入门：逻辑回归应用之Kaggle泰坦尼克之灾 ](https://www.heywhale.com/mw/project/5bfe39b3954d6e0010681cd1)
		
    - [33题实战“泰坦尼克号生存预测”全流程](https://www.heywhale.com/mw/project/5e746b7a98d4a8002d2b0615)
		

## 1. 合并训练集与测试集
　　在进行特征工程的时候，我们不仅需要对训练数据进行处理，还需要同时将测试数据同训练数据一起处理，使得二者具有相同的数据类型和数据分布。

In [19]:
import pandas as pd

train = pd.read_csv('/home/mw/input/wlong9812/train.csv')
test = pd.read_csv('/home/mw/input/wlong9812/test.csv')
train_and_test = train.append(test, sort=False) # 合并训练集与测试集
PassengerId = test['PassengerId']
train_and_test.shape

(1309, 12)

## 2. 缺失值处理
- 对Embarked直接用众数填充；
- 对Fare用均值填充；
- 对Age，建立模型预测；

### 2.1 填充Embarked字段

In [20]:
mode = train_and_test['Embarked'].mode().iloc[0] # 找到众数
train_and_test['Embarked'].fillna(mode, inplace=True)
train_and_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


### 2.2 填充船票Fare字段

In [21]:
train_and_test['Fare'].mean()

33.29547928134557

In [22]:
Fare_mean = train_and_test['Fare'].mean()
train_and_test['Fare'].fillna(Fare_mean, inplace=True)
train_and_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          295 non-null object
Embarked       1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


### 2.3 填充年龄Age字段

　　为尽可能用多的特征去预测Age的值，先对Cabin、Embarked、Name、Sex、Ticket、Pclass等特征进行处理，模型预测见后；

## 3 不同特征字段的数据处理

### 3.1 先对Embarked、Sex以及Pclass等用dummy处理 

In [23]:
# 对分类特征进行编码
cols = ['Embarked', 'Sex', 'Pclass']
train_and_test = pd.get_dummies(train_and_test, columns=cols, prefix_sep='__')
train_and_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 17 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Name           1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          295 non-null object
Embarked__C    1309 non-null uint8
Embarked__Q    1309 non-null uint8
Embarked__S    1309 non-null uint8
Sex__female    1309 non-null uint8
Sex__male      1309 non-null uint8
Pclass__1      1309 non-null uint8
Pclass__2      1309 non-null uint8
Pclass__3      1309 non-null uint8
dtypes: float64(3), int64(3), object(3), uint8(8)
memory usage: 112.5+ KB


### 3.2 票价分级处理
　　我们可以尝试将Fare分桶处理,使用qcut函数。qcut是根据这些值的频率来选择箱子的均匀间隔，每个箱子中含有的数的数量是相同的;

In [24]:
# 临时列
train_and_test['Fare_bin'] = pd.qcut(train_and_test['Fare'], 5)

#编码
train_and_test['Fare_bin_id'] = pd.factorize(train_and_test['Fare_bin'])[0]
fare_bin_dummies_df = pd.get_dummies(train_and_test['Fare_bin_id']).rename(columns=lambda x : 'Fare_' + str(x))
train_and_test = pd.concat([train_and_test, fare_bin_dummies_df], axis=1)
train_and_test.drop(['Fare_bin'], axis=1, inplace=True)

### 3.3 名字处理
　　对名字Name进行处理，提取其特征；

In [25]:
# 提取称呼
train_and_test['Title'] = train_and_test['Name'].apply(lambda x : x.split(',')[1].split('.')[0].strip())
train_and_test['Title']

0          Mr
1         Mrs
2        Miss
3         Mrs
4          Mr
5          Mr
6          Mr
7      Master
8         Mrs
9         Mrs
10       Miss
11       Miss
12         Mr
13         Mr
14       Miss
15        Mrs
16     Master
17         Mr
18        Mrs
19        Mrs
20         Mr
21         Mr
22       Miss
23         Mr
24       Miss
25        Mrs
26         Mr
27         Mr
28       Miss
29         Mr
        ...  
388        Mr
389    Master
390        Mr
391       Mrs
392    Master
393        Mr
394        Mr
395       Mrs
396        Mr
397       Mrs
398        Mr
399        Mr
400      Miss
401        Mr
402      Miss
403        Mr
404        Mr
405        Mr
406        Mr
407        Mr
408      Miss
409      Miss
410      Miss
411       Mrs
412      Miss
413        Mr
414      Dona
415        Mr
416        Mr
417    Master
Name: Title, Length: 1309, dtype: object

In [26]:
# 将各式称呼进行统一化处理
# 头衔映射表
titleDict = {
    "Capt" :      "Officer", 
    "Col"  :      "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}
train_and_test['Title'] = train_and_test['Title'].map(titleDict)
train_and_test['Title'].value_counts()

Mr         757
Miss       262
Mrs        200
Master      61
Officer     23
Royalty      6
Name: Title, dtype: int64

In [27]:
# one_hot编码
train_and_test['Title'] = pd.factorize(train_and_test['Title'])[0]
title_dummies_df = pd.get_dummies(train_and_test['Title'], prefix=train_and_test[['Title']].columns[0])
train_and_test = pd.concat([train_and_test, title_dummies_df], axis=1)
train_and_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 30 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Name           1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          295 non-null object
Embarked__C    1309 non-null uint8
Embarked__Q    1309 non-null uint8
Embarked__S    1309 non-null uint8
Sex__female    1309 non-null uint8
Sex__male      1309 non-null uint8
Pclass__1      1309 non-null uint8
Pclass__2      1309 non-null uint8
Pclass__3      1309 non-null uint8
Fare_bin_id    1309 non-null int64
Fare_0         1309 non-null uint8
Fare_1         1309 non-null uint8
Fare_2         1309 non-null uint8
Fare_3         1309 non-null uint8
Fare_4         1309 non-null uint8
Title          1309 non-null int64
Title_0        1309 non-null uint8
Title_1     

In [28]:
# 提取长度特征
train_and_test['Name_length'] = train_and_test['Name'].apply(len)
train_and_test['Name_length']

0      23
1      51
2      22
3      44
4      24
5      16
6      23
7      30
8      49
9      35
10     31
11     24
12     30
13     27
14     36
15     32
16     20
17     28
18     55
19     23
20     20
21     21
22     27
23     28
24     29
25     57
26     23
27     30
28     29
29     19
       ..
388    20
389    27
390    26
391    46
392    29
393    20
394    24
395    46
396    20
397    63
398    30
399    24
400    23
401    15
402    30
403    22
404    28
405    44
406    25
407    26
408    31
409    25
410    22
411    47
412    30
413    18
414    28
415    28
416    19
417    24
Name: Name_length, Length: 1309, dtype: int64

### 3.4 Cabin处理
　　Cabin缺失值过多，将其分为有无两类，进行编码，如果缺失，即为0，否则为1;

In [29]:
train_and_test.loc[train_and_test.Cabin.isnull(), 'Cabin'] = 'U0'
train_and_test['Cabin'] = train_and_test['Cabin'].apply(lambda x : 0 if x == 'U0' else 1)
train_and_test['Cabin']

0      0
1      1
2      0
3      1
4      0
5      0
6      1
7      0
8      0
9      0
10     1
11     1
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     1
22     0
23     1
24     0
25     0
26     0
27     1
28     0
29     0
      ..
388    0
389    0
390    1
391    1
392    0
393    0
394    0
395    1
396    0
397    1
398    0
399    0
400    1
401    0
402    0
403    0
404    1
405    1
406    0
407    1
408    0
409    0
410    0
411    1
412    0
413    0
414    1
415    0
416    0
417    0
Name: Cabin, Length: 1309, dtype: int64

### 3.5 Ticket处理
　　Ticket有字母和数字之分，对于不同的字母，可能在很大程度上就意味着船舱等级或者不同船舱的位置，也会对Survived产生一定的影响，所以我们将Ticket中的字母分开，为数字的部分则分为一类。

In [30]:
train_and_test['Ticket_Letter'] = train_and_test['Ticket'].str.split().str[0]
train_and_test['Ticket_Letter'] = train_and_test['Ticket_Letter'].apply(lambda x : 'U0' if x.isnumeric() else x)

# 将Ticket_Letter factorize
train_and_test['Ticket_Letter'] = pd.factorize(train_and_test['Ticket_Letter'])[0]
train_and_test['Ticket_Letter']

0       0
1       1
2       2
3       3
4       3
5       3
6       3
7       3
8       3
9       3
10      4
11      3
12      5
13      3
14      3
15      3
16      3
17      3
18      3
19      3
20      3
21      3
22      3
23      3
24      3
25      3
26      3
27      3
28      3
29      3
       ..
388     3
389     3
390     3
391     1
392     6
393     6
394     3
395     3
396     3
397     3
398     3
399     3
400     3
401     3
402     3
403     3
404     3
405    22
406     3
407     3
408     3
409    21
410     3
411     3
412     3
413    24
414     1
415    21
416     3
417     3
Name: Ticket_Letter, Length: 1309, dtype: int64

## 4. 利用随机森林预测Age缺失值

In [31]:
from sklearn.ensemble import RandomForestRegressor  # 随机森林回归

missing_age = train_and_test.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1) # 去除字符串类型的字段
missing_age_train = missing_age[missing_age['Age'].notnull()]
missing_age_test = missing_age[missing_age['Age'].isnull()]

X_train = missing_age_train.iloc[:,1:]
y_train = missing_age_train.iloc[:,0]
X_test = missing_age_test.iloc[:,1:]

rfr = RandomForestRegressor(n_estimators=1000, n_jobs=-1)
rfr.fit(X_train, y_train)
y_predict = rfr.predict(X_test)
train_and_test.loc[train_and_test['Age'].isnull(), 'Age'] = y_predict
train_and_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 32 columns):
PassengerId      1309 non-null int64
Survived         891 non-null float64
Name             1309 non-null object
Age              1309 non-null float64
SibSp            1309 non-null int64
Parch            1309 non-null int64
Ticket           1309 non-null object
Fare             1309 non-null float64
Cabin            1309 non-null int64
Embarked__C      1309 non-null uint8
Embarked__Q      1309 non-null uint8
Embarked__S      1309 non-null uint8
Sex__female      1309 non-null uint8
Sex__male        1309 non-null uint8
Pclass__1        1309 non-null uint8
Pclass__2        1309 non-null uint8
Pclass__3        1309 non-null uint8
Fare_bin_id      1309 non-null int64
Fare_0           1309 non-null uint8
Fare_1           1309 non-null uint8
Fare_2           1309 non-null uint8
Fare_3           1309 non-null uint8
Fare_4           1309 non-null uint8
Title            1309 non-null int64

## 5. 各特征与Survived的相关系数排序

In [32]:
# 根据生存情况与其他各特征的相关系数，按系数倒序排序，筛选出重要特征 -- 重要特征
train_and_test.corr()['Survived'].abs().sort_values(ascending=False)

Survived         1.000000
Title_0          0.549199
Sex__male        0.543351
Sex__female      0.543351
Title            0.377155
Title_1          0.344935
Title_2          0.332795
Name_length      0.332350
Pclass__3        0.322308
Cabin            0.316912
Pclass__1        0.285904
Fare_1           0.263007
Fare             0.257307
Fare_2           0.191707
Fare_0           0.171109
Embarked__C      0.168240
Embarked__S      0.149683
Pclass__2        0.093349
Title_3          0.085221
Ticket_Letter    0.082068
Parch            0.081629
Age              0.080912
Fare_bin_id      0.071787
Fare_4           0.064843
Fare_3           0.040812
SibSp            0.035322
Title_4          0.033391
Title_5          0.031316
PassengerId      0.005007
Embarked__Q      0.003650
Name: Survived, dtype: float64

## 6. 保存特征处理后的数据

In [33]:
train_and_test.to_csv('/home/mw/input/wlong9812/经过特征工程处理后的数据.csv', index=None)

## 7. 小结

- 特征工程这一章主要做了以下工作：
    - 合并训练集和测试集
        - 为了使二者具有相同的数据类型和数据分布；
    - 缺失值处理：
        - Embarked:众数填充；
        - Fare:平均值填充；
        - Age:随机森林预测填充；
    - 各特征字段的数据处理：
        - Embarked,Sex,Pclass: 直接dummy编码；
        - Fare: 先分桶处理，再dummy编码；
        - Name: 先提取称呼，再对称呼进行人群分类，最后dummy处理；
        - cabin：缺失值较多，根据是否缺失划分类别，缺失为0，否则为1；
        - Ticket:只保留其中字母，并对字母进行数字转换；
    - 随机森林建模预测Age缺失值；
    - 对各特征与生存与否进行了相关系数大小排序；

----

　　好了，本小章节主要谈到泰坦尼克号数据的特征工程处理，后续就是建模预测部分了，建模预测打算分两部分，一部分只是简单涉及一些算法，参数全部默认；另一部分，会加些算法调参、优化以及复杂模型等，这几天就会安排上！
	
　　如果本文有存在不足的地方，欢迎大家在评论区留言，我会继续完善的；
  
----